# Debug unsplittable molecules

In [1]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")
sys.path.insert(0,"..")

import os
import json

import random
import time as time
import pandas as pd
import numpy as np
from pathlib import Path

from rmgpy.molecule.group import Group
from rmgpy.data.base import Entry
from rmgpy.data.thermo import ThermoData

from tree.thermo import ThermoGroups, average_thermo_data
from tree.utils import make_mol
from tree.parameters import Ts

# Make data to debug

In [6]:
mol1 = make_mol("C=C(CO[O])CC(C)=O", label=True)
mol2 = make_mol("C=C(O)CC(=C)CO[O]", label=True)

mols = [mol1, mol2]
corrections = [
    ThermoData(
        Tdata=(Ts, "K"),
        Cpdata=(Ts, "cal/mol/K", "+|-", Ts),
        H298=(Ts[0], "kcal/mol", "+|-", Ts[0]),
        S298=(Ts[0], "cal/mol/K", "+|-", Ts[0]),
    ),
    ThermoData(
        Tdata=(Ts, "K"),
        Cpdata=(Ts, "cal/mol/K", "+|-", Ts),
        H298=(Ts[0], "kcal/mol", "+|-", Ts[0]),
        S298=(Ts[0], "cal/mol/K", "+|-", Ts[0]),
    ),
]

mols_corrections_all = list(zip(mols, corrections))

In [7]:
print(mol1.to_adjacency_list())

multiplicity 2
1    C u0 p0 c0 {2,D} {10,S} {11,S}
2    C u0 p0 c0 {1,D} {3,S} {6,S}
3    C u0 p0 c0 {2,S} {4,S} {12,S} {13,S}
4    O u0 p2 c0 {3,S} {5,S}
5  * O u1 p2 c0 {4,S}
6    C u0 p0 c0 {2,S} {7,S} {14,S} {15,S}
7    C u0 p0 c0 {6,S} {8,S} {9,D}
8    C u0 p0 c0 {7,S} {16,S} {17,S} {18,S}
9    O u0 p2 c0 {7,D}
10   H u0 p0 c0 {1,S}
11   H u0 p0 c0 {1,S}
12   H u0 p0 c0 {3,S}
13   H u0 p0 c0 {3,S}
14   H u0 p0 c0 {6,S}
15   H u0 p0 c0 {6,S}
16   H u0 p0 c0 {8,S}
17   H u0 p0 c0 {8,S}
18   H u0 p0 c0 {8,S}



# Generate tree

In [8]:
n_jobs = 4
# save_dir = "split-random_run-test"
# os.makedirs(f"../models/{save_dir}/", exist_ok=True)
mols_corrections = mols_corrections_all

In [9]:
#clean tree
tree = ThermoGroups(
    label="radical",
    name="HBI correction",
)

tree.entries["Root"] = Entry(
    index = 0,
    label = "Root",
    item = Group().from_adjacency_list(f"""1 * R u[1,2,3,4]"""),
    data = None,
    data_count = 0,
    parent = None,
)
tree.entries["RJ1"] = Entry(
    index = 0,
    label = "RJ1",
    item = Group().from_adjacency_list(f"""1 * R u1"""),
    data = None,
    data_count = 0,
    parent = tree.entries["Root"],
)

tree.entries["Root"].children = [tree.entries["RJ1"]]
tree.top = [tree.entries["Root"]]


start = time.time()
template_mol_map_exact = tree.generate_tree(mols_corrections=mols_corrections, obj=None, Ts=None, nprocs=1, min_splitable_entry_num=2,
                                          min_mols_corrections_to_spawn=20, max_batch_size=np.inf, outlier_fraction=0.02, stratum_num=8,
                                          new_fraction_threshold_to_reopt_node=0.25, extension_iteration_max=2, extension_iteration_item_cap=100, 
                                            min_mols_corrections_to_split=1, n_jobs=n_jobs)
end = time.time()
print("Tree generation:")
print(end-start)

tree.check_tree()

start = time.time()
template_mol_map = tree.get_molecule_matches(mols_corrections=mols_corrections,
                                                     exact_matches_only=False, n_jobs=n_jobs)
end = time.time()
print("Mol mapping:")
print(end-start)

tree.regularize(template_mol_map)

start = time.time()
tree.make_corrections_from_template_mol_map(template_mol_map, n_jobs=n_jobs)
end = time.time()
print("Make corrections:")
print(end-start)

tree.check_tree()

# def add_children(old_node, new_node):
#     for old_child in old_node.children:
#         tree.entries[old_child.label] = deepcopy(old_child)
#         tree.entries[old_child.label].parent = new_node
#         new_node.children.append(tree.entries[old_child.label])
#         add_children(old_child, tree.entries[old_child.label])
# tree.entries["RJ2_triplet"] = deepcopy(thermo_database.groups["radical"].entries["RJ2_triplet"])
# tree.entries["RJ2_triplet"].parent = tree.entries["Root"]
# tree.entries["Root"].children.append(tree.entries["RJ2_triplet"])
# add_children(thermo_database.groups["radical"].entries["RJ2_triplet"], tree.entries["RJ2_triplet"])

# tree.entries["RJ3"] = deepcopy(thermo_database.groups["radical"].entries["RJ3"])
# tree.entries["RJ3"].parent = tree.entries["Root"]
# tree.entries["Root"].children.append(tree.entries["RJ3"])
# add_children(thermo_database.groups["radical"].entries["RJ3"], tree.entries["RJ3"])

# tree.save(f"../models/{save_dir}/tree.py")

INFO:root:Getting molecule matches...
[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s finished
INFO:root:Number of datapoints: 2.0
INFO:root:Tree size: 2
ERROR:root:split violation
ERROR:root:parent
ERROR:root:1 * R u1

ERROR:root:0
ERROR:root:[[<AtomType "O">], [<AtomType "O">]]
ERROR:root:[[], []]
ERROR:root:bonds:
ERROR:root:<Molecule "C=C(O)CC(=C)CO[O]">
ERROR:root:C=C(O)CC(=C)CO[O]
ERROR:root:multiplicity 2
1    O u0 p2 c0 {3,S} {5,S}
2    O u0 p2 c0 {7,S} {18,S}
3  * O u1 p2 c0 {1,S}
4    C u0 p0 c0 {6,S} {7,S} {10,S} {11,S}
5    C u0 p0 c0 {1,S} {6,S} {12,S} {13,S}
6    C u0 p0 c0 {4,S} {5,S} {8,D}
7    C u0 p0 c0 {2,S} {4,S} {9,D}
8    C u0 p0 c0 {6,D} {16,S} {17,S}
9    C u0 p0 c0 {7,D} {14,S} {15,S}
10   H u0 p0 c0 {4,S}
11   H u0 p0 c0 {4,S}
12   H u0 p0 c0 {5,S}
13   H u0 p0 c0 {5,S}
14   H u0 p0 c

Tree generation:
1.6355047225952148


[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    0.0s finished


Mol mapping:
0.20470309257507324
Make corrections:
0.2272946834564209


[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.0s finished
